# **Chapter 42: Unit Testing Frameworks**

---

## **42.1 JUnit (Java)**

### **42.1.1 Introduction to JUnit**

JUnit is the de facto standard testing framework for Java applications. It provides annotations, assertions, and test runners that enable developers to write repeatable and automated unit tests. JUnit 5 (Jupiter) represents the latest evolution with enhanced features and modular architecture.

**JUnit Architecture:**
```
JUnit 5 = JUnit Platform + JUnit Jupiter + JUnit Vintage
- Platform: Test engine API for launching tests
- Jupiter: New programming model with annotations
- Vintage: Backward compatibility for JUnit 3/4
```

### **42.1.2 Annotations and Assertions**

```java
// Basic JUnit Test Structure
import org.junit.jupiter.api.*;
import org.junit.jupiter.params.ParameterizedTest;
import org.junit.jupiter.params.provider.*;
import static org.junit.jupiter.api.Assertions.*;
import static org.junit.jupiter.api.Assumptions.*;

import java.time.Duration;
import java.util.Arrays;
import java.util.List;
import java.util.stream.Stream;

class CalculatorTest {
    
    private Calculator calculator;
    
    // Lifecycle Hooks
    @BeforeAll
    static void setUpBeforeAll() {
        System.out.println("Runs once before all tests");
        // Initialize expensive resources (database connections, etc.)
    }
    
    @AfterAll
    static void tearDownAfterAll() {
        System.out.println("Runs once after all tests");
        // Clean up resources
    }
    
    @BeforeEach
    void setUp() {
        System.out.println("Runs before each test");
        calculator = new Calculator();
    }
    
    @AfterEach
    void tearDown() {
        System.out.println("Runs after each test");
        calculator = null;
    }
    
    // Basic Assertions
    @Test
    @DisplayName("Test addition with positive numbers")
    void testAddition() {
        // Arrange
        int a = 5;
        int b = 3;
        int expected = 8;
        
        // Act
        int result = calculator.add(a, b);
        
        // Assert
        assertEquals(expected, result, "Addition should return sum");
        assertNotEquals(10, result);
        assertTrue(result > 0);
        assertFalse(result < 0);
    }
    
    @Test
    void testSubtraction() {
        assertEquals(2, calculator.subtract(5, 3));
        assertEquals(-2, calculator.subtract(3, 5));
        assertNotEquals(0, calculator.subtract(1, 1));
    }
    
    // Testing Exceptions
    @Test
    void testDivisionByZero() {
        Exception exception = assertThrows(ArithmeticException.class, 
            () -> calculator.divide(10, 0));
        
        assertEquals("/ by zero", exception.getMessage());
        
        assertDoesNotThrow(() -> calculator.divide(10, 2));
    }
    
    // Testing with Timeouts
    @Test
    void testPerformance() {
        assertTimeout(Duration.ofMillis(100), () -> {
            calculator.complexOperation(1000);
        });
        
        // Preemptively fails if timeout exceeded
        assertTimeoutPreemptively(Duration.ofMillis(50), () -> {
            calculator.complexOperation(100);
        });
    }
    
    // Multiple Assertions
    @Test
    void testMultipleAssertions() {
        Calculator.Result result = calculator.calculate(10, 5);
        
        assertAll("Calculator operations",
            () -> assertEquals(15, result.getSum()),
            () -> assertEquals(5, result.getDifference()),
            () -> assertEquals(50, result.getProduct()),
            () -> assertEquals(2.0, result.getQuotient(), 0.001)
        );
    }
    
    // Assumptions (Conditional Test Execution)
    @Test
    void testOnlyOnDevelopment() {
        assumeTrue("dev".equals(System.getProperty("environment")));
        // Test will be skipped if assumption fails
        assertEquals(4, calculator.add(2, 2));
    }
    
    // Nested Tests
    @Nested
    @DisplayName("When calculator is in scientific mode")
    class ScientificModeTests {
        
        @BeforeEach
        void enableScientificMode() {
            calculator.setMode(Calculator.Mode.SCIENTIFIC);
        }
        
        @Test
        void testPower() {
            assertEquals(8.0, calculator.power(2, 3));
        }
        
        @Test
        void testSquareRoot() {
            assertEquals(3.0, calculator.sqrt(9));
            assertThrows(IllegalArgumentException.class, 
                () -> calculator.sqrt(-1));
        }
    }
    
    // Parameterized Tests
    @ParameterizedTest
    @ValueSource(ints = {1, 2, 3, 4, 5})
    void testIsPositive(int number) {
        assertTrue(calculator.isPositive(number));
    }
    
    @ParameterizedTest
    @CsvSource({
        "1, 1, 2",
        "2, 3, 5",
        "5, -2, 3",
        "0, 0, 0"
    })
    void testAdditionWithCsvSource(int a, int b, int expected) {
        assertEquals(expected, calculator.add(a, b));
    }
    
    @ParameterizedTest
    @MethodSource("provideNumbersForMultiplication")
    void testMultiplication(int a, int b, int expected) {
        assertEquals(expected, calculator.multiply(a, b));
    }
    
    static Stream<Arguments> provideNumbersForMultiplication() {
        return Stream.of(
            Arguments.of(2, 3, 6),
            Arguments.of(5, 5, 25),
            Arguments.of(10, 0, 0)
        );
    }
    
    @ParameterizedTest
    @EnumSource(Calculator.Operation.class)
    void testAllOperations(Calculator.Operation operation) {
        assertNotNull(calculator.perform(operation, 10, 5));
    }
}

// Calculator Class (System Under Test)
class Calculator {
    
    public enum Mode { BASIC, SCIENTIFIC }
    public enum Operation { ADD, SUBTRACT, MULTIPLY, DIVIDE }
    
    public static class Result {
        private int sum;
        private int difference;
        private int product;
        private double quotient;
        
        public Result(int sum, int diff, int prod, double quot) {
            this.sum = sum;
            this.difference = diff;
            this.product = prod;
            this.quotient = quot;
        }
        
        // Getters...
    }
    
    private Mode mode = Mode.BASIC;
    
    public void setMode(Mode mode) { this.mode = mode; }
    
    public int add(int a, int b) { return a + b; }
    public int subtract(int a, int b) { return a - b; }
    public int multiply(int a, int b) { return a * b; }
    
    public double divide(int a, int b) {
        if (b == 0) throw new ArithmeticException("/ by zero");
        return (double) a / b;
    }
    
    public Result calculate(int a, int b) {
        return new Result(
            add(a, b),
            subtract(a, b),
            multiply(a, b),
            divide(a, b)
        );
    }
    
    public double power(double base, double exponent) {
        return Math.pow(base, exponent);
    }
    
    public double sqrt(double value) {
        if (value < 0) throw new IllegalArgumentException("Cannot sqrt negative number");
        return Math.sqrt(value);
    }
    
    public boolean isPositive(int number) {
        return number > 0;
    }
    
    public void complexOperation(int iterations) {
        for (int i = 0; i < iterations; i++) {
            Math.sqrt(Math.pow(i, 2));
        }
    }
    
    public Object perform(Operation operation, int a, int b) {
        switch (operation) {
            case ADD: return add(a, b);
            case SUBTRACT: return subtract(a, b);
            case MULTIPLY: return multiply(a, b);
            case DIVIDE: return divide(a, b);
            default: throw new UnsupportedOperationException();
        }
    }
}
```

### **42.1.3 Mocking with Mockito**

```java
// Mockito for Dependency Isolation
import org.junit.jupiter.api.Test;
import org.junit.jupiter.api.extension.ExtendWith;
import org.mockito.*;
import org.mockito.junit.jupiter.MockitoExtension;
import static org.mockito.Mockito.*;
import static org.junit.jupiter.api.Assertions.*;

import java.util.Arrays;
import java.util.List;
import java.util.Optional;

@ExtendWith(MockitoExtension.class)
class UserServiceTest {
    
    @Mock
    private UserRepository userRepository;
    
    @Mock
    private EmailService emailService;
    
    @InjectMocks
    private UserService userService;  // Automatically injects mocks
    
    @Captor
    private ArgumentCaptor<User> userCaptor;
    
    @Test
    void testFindUserById() {
        // Arrange
        Long userId = 1L;
        User expectedUser = new User(userId, "john@example.com", "John Doe");
        
        when(userRepository.findById(userId)).thenReturn(Optional.of(expectedUser));
        
        // Act
        Optional<User> result = userService.findUserById(userId);
        
        // Assert
        assertTrue(result.isPresent());
        assertEquals(expectedUser, result.get());
        verify(userRepository).findById(userId);
        verifyNoMoreInteractions(userRepository);
    }
    
    @Test
    void testRegisterNewUser() {
        // Arrange
        String email = "newuser@example.com";
        String name = "New User";
        
        when(userRepository.findByEmail(email)).thenReturn(Optional.empty());
        when(userRepository.save(any(User.class))).thenAnswer(invocation -> {
            User user = invocation.getArgument(0);
            user.setId(2L);
            return user;
        });
        
        // Act
        User result = userService.registerUser(email, name);
        
        // Assert
        assertNotNull(result);
        assertEquals(2L, result.getId());
        assertEquals(email, result.getEmail());
        
        verify(userRepository).findByEmail(email);
        verify(userRepository).save(any(User.class));
        verify(emailService).sendWelcomeEmail(email, name);
    }
    
    @Test
    void testRegisterExistingUser() {
        // Arrange
        String email = "existing@example.com";
        User existingUser = new User(1L, email, "Existing");
        
        when(userRepository.findByEmail(email)).thenReturn(Optional.of(existingUser));
        
        // Act & Assert
        assertThrows(UserAlreadyExistsException.class, 
            () -> userService.registerUser(email, "Name"));
        
        verify(userRepository, never()).save(any(User.class));
        verify(emailService, never()).sendWelcomeEmail(anyString(), anyString());
    }
    
    @Test
    void testUpdateUserEmail() {
        // Arrange
        Long userId = 1L;
        String newEmail = "new@example.com";
        User existingUser = new User(userId, "old@example.com", "User");
        
        when(userRepository.findById(userId)).thenReturn(Optional.of(existingUser));
        
        // Act
        userService.updateEmail(userId, newEmail);
        
        // Assert
        verify(userRepository).findById(userId);
        
        // Capture the saved user
        verify(userRepository).save(userCaptor.capture());
        User savedUser = userCaptor.getValue();
        
        assertEquals(newEmail, savedUser.getEmail());
    }
    
    @Test
    void testBulkUserCreation() {
        // Arrange
        List<UserData> userDataList = Arrays.asList(
            new UserData("user1@example.com", "User 1"),
            new UserData("user2@example.com", "User 2"),
            new UserData("user3@example.com", "User 3")
        );
        
        // Stub void method with different behaviors
        doNothing().when(emailService).sendWelcomeEmail(anyString(), anyString());
        
        // Act
        List<User> results = userService.createUsersBulk(userDataList);
        
        // Assert
        assertEquals(3, results.size());
        
        // Verify interactions with specific arguments
        verify(userRepository, times(3)).save(any(User.class));
        verify(emailService, times(3)).sendWelcomeEmail(anyString(), anyString());
        
        // Verify order of interactions
        InOrder inOrder = inOrder(userRepository, emailService);
        inOrder.verify(userRepository, times(3)).save(any(User.class));
        inOrder.verify(emailService, times(3)).sendWelcomeEmail(anyString(), anyString());
    }
    
    @Test
    void testSpyExample() {
        // Spy on real object (partial mocking)
        UserService realService = new UserService(userRepository, emailService);
        UserService spyService = spy(realService);
        
        // Mock only specific method
        doReturn(true).when(spyService).isValidEmail(anyString());
        
        // Call real method
        when(userRepository.save(any())).thenReturn(new User(1L, "test@example.com", "Test"));
        
        User result = spyService.registerUser("test@example.com", "Test");
        
        assertNotNull(result);
        verify(spyService).isValidEmail("test@example.com");
    }
    
    @Test
    void testAnswerInterface() {
        // Custom answer for dynamic responses
        when(userRepository.save(any(User.class))).thenAnswer(invocation -> {
            User user = invocation.getArgument(0);
            user.setId(ThreadLocalRandom.current().nextLong(1000));
            return user;
        });
        
        User user = userService.registerUser("test@example.com", "Test");
        
        assertNotNull(user.getId());
    }
    
    @Test
    void testResetMocks() {
        // Reset mock state
        userService.findUserById(1L);
        
        reset(userRepository);
        
        verifyNoInteractions(userRepository);
    }
}

// Supporting Classes
class User {
    private Long id;
    private String email;
    private String name;
    
    // Constructor, getters, setters...
    public User(Long id, String email, String name) {
        this.id = id;
        this.email = email;
        this.name = name;
    }
    
    public Long getId() { return id; }
    public void setId(Long id) { this.id = id; }
    public String getEmail() { return email; }
    public void setEmail(String email) { this.email = email; }
    public String getName() { return name; }
}

class UserData {
    private String email;
    private String name;
    public UserData(String email, String name) {
        this.email = email;
        this.name = name;
    }
    // Getters...
}

interface UserRepository {
    Optional<User> findById(Long id);
    Optional<User> findByEmail(String email);
    User save(User user);
    void delete(User user);
}

interface EmailService {
    void sendWelcomeEmail(String email, String name);
    void sendPasswordReset(String email, String token);
}

class UserService {
    private final UserRepository userRepository;
    private final EmailService emailService;
    
    public UserService(UserRepository userRepository, EmailService emailService) {
        this.userRepository = userRepository;
        this.emailService = emailService;
    }
    
    public Optional<User> findUserById(Long id) {
        return userRepository.findById(id);
    }
    
    public User registerUser(String email, String name) {
        if (!isValidEmail(email)) {
            throw new IllegalArgumentException("Invalid email");
        }
        
        Optional<User> existing = userRepository.findByEmail(email);
        if (existing.isPresent()) {
            throw new UserAlreadyExistsException("User with email " + email + " already exists");
        }
        
        User user = new User(null, email, name);
        User saved = userRepository.save(user);
        emailService.sendWelcomeEmail(email, name);
        return saved;
    }
    
    public void updateEmail(Long userId, String newEmail) {
        User user = userRepository.findById(userId)
            .orElseThrow(() -> new UserNotFoundException("User not found"));
        
        if (!isValidEmail(newEmail)) {
            throw new IllegalArgumentException("Invalid email");
        }
        
        user.setEmail(newEmail);
        userRepository.save(user);
    }
    
    public List<User> createUsersBulk(List<UserData> userDataList) {
        return userDataList.stream()
            .map(data -> registerUser(data.getEmail(), data.getName()))
            .toList();
    }
    
    public boolean isValidEmail(String email) {
        return email != null && email.contains("@") && email.contains(".");
    }
}

class UserAlreadyExistsException extends RuntimeException {
    public UserAlreadyExistsException(String message) { super(message); }
}

class UserNotFoundException extends RuntimeException {
    public UserNotFoundException(String message) { super(message); }
}
```

### **42.1.4 Advanced JUnit Features**

```java
// Test Interfaces and Default Methods
interface TestLifecycleLogger {
    
    @BeforeEach
    default void beforeEachTest(TestInfo testInfo) {
        System.out.println("Starting: " + testInfo.getDisplayName());
    }
    
    @AfterEach
    default void afterEachTest(TestInfo testInfo) {
        System.out.println("Finished: " + testInfo.getDisplayName());
    }
}

class AdvancedCalculatorTest implements TestLifecycleLogger {
    
    private Calculator calculator;
    
    @BeforeEach
    void setUp(TestInfo testInfo) {
        calculator = new Calculator();
        System.out.println("Setup for: " + testInfo.getDisplayName());
    }
    
    @Test
    void testWithTags() {
        assertEquals(4, calculator.add(2, 2));
    }
}

// Dynamic Tests (Runtime Test Generation)
class DynamicTests {
    
    private final Calculator calculator = new Calculator();
    
    @TestFactory
    List<DynamicTest> dynamicTestsFromCollection() {
        return Arrays.asList(
            DynamicTest.dynamicTest("Addition Test", 
                () -> assertEquals(5, calculator.add(2, 3))),
            DynamicTest.dynamicTest("Multiplication Test",
                () -> assertEquals(6, calculator.multiply(2, 3)))
        );
    }
    
    @TestFactory
    Stream<DynamicTest> dynamicTestsFromStream() {
        List<TestCase> testCases = Arrays.asList(
            new TestCase(2, 3, 5),
            new TestCase(5, 5, 10),
            new TestCase(10, -2, 8)
        );
        
        return testCases.stream()
            .map(tc -> DynamicTest.dynamicTest(
                String.format("Testing %d + %d = %d", tc.a, tc.b, tc.expected),
                () -> assertEquals(tc.expected, calculator.add(tc.a, tc.b))
            ));
    }
    
    record TestCase(int a, int b, int expected) {}
}

// Test Repetition
class RepeatedTests {
    
    @RepeatedTest(value = 5, name = "{displayName} - repetition {currentRepetition}/{totalRepetitions}")
    @DisplayName("Random value test")
    void testRandomValues(RepetitionInfo repetitionInfo) {
        int random = new Random().nextInt(100);
        assertTrue(random >= 0 && random < 100);
        
        System.out.printf("Repetition %d of %d%n",
            repetitionInfo.getCurrentRepetition(),
            repetitionInfo.getTotalRepetitions());
    }
}

// Test Templates
class TestTemplatesDemo {
    
    @TestTemplate
    @ExtendWith(MyTestTemplateInvocationContextProvider.class)
    void testTemplate(String parameter) {
        assertNotNull(parameter);
    }
}

// Conditional Test Execution
class ConditionalTests {
    
    @Test
    @EnabledOnOs({OS.MAC, OS.LINUX})
    void testOnUnix() {
        // Only runs on Mac or Linux
    }
    
    @Test
    @DisabledOnOs(OS.WINDOWS)
    void testNotOnWindows() {
        // Runs everywhere except Windows
    }
    
    @Test
    @EnabledOnJre({JRE.JAVA_11, JRE.JAVA_17})
    void testOnlyOnJava11Or17() {
        // Only runs on specified JRE versions
    }
    
    @Test
    @DisabledForJreRange(min = JRE.JAVA_8, max = JRE.JAVA_11)
    void testNotOnJava8To11() {
        // Disabled for Java 8 through 11
    }
    
    @Test
    @EnabledIfSystemProperty(named = "os.arch", matches = ".*64.*")
    void testOnlyOn64BitArch() {
        // Only on 64-bit architectures
    }
    
    @Test
    @EnabledIfEnvironmentVariable(named = "ENV", matches = "dev|test")
    void testOnlyInDevOrTestEnvironment() {
        // Only when ENV is dev or test
    }
}
```

---

## **42.2 pytest (Python)**

### **42.2.1 Introduction to pytest**

pytest is a mature full-featured Python testing framework that makes it easy to write simple and scalable tests. It requires minimal boilerplate code and includes powerful features like fixtures, parameterization, and plugins.

### **42.2.2 Basic Test Structure**

```python
# test_calculator.py
import pytest
import math
from datetime import datetime
from typing import List, Tuple

# Simple test functions
def test_addition():
    """Test basic addition"""
    result = add(5, 3)
    assert result == 8
    assert isinstance(result, int)

def test_subtraction():
    """Test subtraction with negative results"""
    assert subtract(5, 3) == 2
    assert subtract(3, 5) == -2

# Test class grouping
class TestCalculator:
    """Group related tests in a class"""
    
    def setup_method(self, method):
        """Setup before each test method (traditional xUnit style)"""
        print(f"Setting up {method.__name__}")
        self.calculator = Calculator()
    
    def teardown_method(self, method):
        """Teardown after each test method"""
        print(f"Tearing down {method.__name__}")
        self.calculator = None
    
    def test_multiplication(self):
        assert self.calculator.multiply(4, 3) == 12
    
    def test_division(self):
        assert self.calculator.divide(10, 2) == 5.0
    
    def test_divide_by_zero(self):
        with pytest.raises(ValueError, match="Cannot divide by zero"):
            self.calculator.divide(10, 0)
    
    def test_multiple_operations(self):
        result = self.calculator.calculate(10, 5)
        assert result['sum'] == 15
        assert result['difference'] == 5
        assert result['product'] == 50
        assert result['quotient'] == 2.0

# Assertion introspection
def test_assertion_details():
    """pytest provides detailed assertion failure information"""
    expected = [1, 2, 3, 4, 5]
    actual = [1, 2, 3, 4, 6]
    
    # This will show exactly which elements differ
    assert actual == expected, "Lists should be equal"
    
    # Dictionary comparison
    dict1 = {'a': 1, 'b': 2, 'c': 3}
    dict2 = {'a': 1, 'b': 3, 'c': 3}
    assert dict1 == dict2

# Approximate comparisons for floating point
def test_floating_point():
    """Testing floating point numbers with tolerance"""
    result = 0.1 + 0.2
    assert result == pytest.approx(0.3)
    assert result == pytest.approx(0.3, rel=1e-6)  # Relative tolerance
    assert 3.14159 == pytest.approx(3.14, abs=0.01)  # Absolute tolerance

# Exception testing
def test_exception_details():
    """Test exception messages and types"""
    with pytest.raises(ValueError) as exc_info:
        raise ValueError("Invalid input")
    
    assert str(exc_info.value) == "Invalid input"
    assert exc_info.type is ValueError

# Warning testing
def test_warning():
    """Test that code produces expected warnings"""
    with pytest.warns(UserWarning, match="deprecated"):
        import warnings
        warnings.warn("This function is deprecated", UserWarning)

# Marking tests
@pytest.mark.slow
def test_expensive_operation():
    """Mark slow tests to potentially skip"""
    import time
    time.sleep(2)
    assert True

@pytest.mark.skip(reason="Not implemented yet")
def test_not_implemented():
    """Skip test with reason"""
    assert False

@pytest.mark.skipif(sys.version_info < (3, 8), 
                    reason="Requires Python 3.8+")
def test_new_feature():
    """Skip test conditionally"""
    assert True

@pytest.mark.xfail(reason="Known issue, will fix later")
def test_expected_failure():
    """Mark test as expected to fail"""
    assert False  # This will show as XFAIL, not FAIL

@pytest.mark.xfail(strict=True)
def test_expected_failure_strict():
    """Strict XFAIL - if it passes, test fails"""
    assert False

# Custom markers
@pytest.mark.smoke
@pytest.mark.api
def test_api_endpoint():
    """Multiple markers can be applied"""
    assert True

# Running with markers:
# pytest -m "smoke"           # Run only smoke tests
# pytest -m "not slow"        # Exclude slow tests
# pytest -m "api or unit"     # Run API or unit tests

# Parameterized testing
@pytest.mark.parametrize("a,b,expected", [
    (1, 1, 2),
    (2, 3, 5),
    (5, -2, 3),
    (0, 0, 0),
    (100, 200, 300)
])
def test_parameterized_addition(a, b, expected):
    """Run same test with multiple data sets"""
    assert add(a, b) == expected

# Parameterized with IDs for better reporting
@pytest.mark.parametrize(
    "a,b,expected",
    [
        (2, 3, 5),
        (5, 5, 10),
        (10, -5, 5),
    ],
    ids=["positive", "same", "negative"]
)
def test_addition_with_ids(a, b, expected):
    assert add(a, b) == expected

# Multiple parameterization levels
@pytest.mark.parametrize("a", [1, 2, 3])
@pytest.mark.parametrize("b", [4, 5, 6])
def test_cartesian_product(a, b):
    """Tests all combinations of a and b"""
    result = add(a, b)
    assert result == a + b

# Fixtures
@pytest.fixture
def calculator():
    """Provide a calculator instance for tests"""
    print("Setting up calculator")
    calc = Calculator()
    yield calc  # This is where the test runs
    print("Cleaning up calculator")
    calc.clear()

def test_with_fixture(calculator):
    """Use fixture as parameter"""
    assert calculator.add(5, 3) == 8

# Fixture scopes
@pytest.fixture(scope="session")
def db_connection():
    """Create once per test session"""
    print("Connecting to database")
    conn = DatabaseConnection("test.db")
    yield conn
    print("Closing database connection")
    conn.close()

@pytest.fixture(scope="module")
def module_data():
    """Create once per test module"""
    return {"initialized": datetime.now()}

@pytest.fixture(scope="class")
def class_data():
    """Create once per test class"""
    return {"shared": "data"}

@pytest.fixture(scope="function", autouse=True)
def auto_timing():
    """Automatically used in every test"""
    start = datetime.now()
    yield
    duration = (datetime.now() - start).total_seconds()
    print(f"Test took {duration:.3f} seconds")

# Fixture dependencies
@pytest.fixture
def user(db_connection):
    """Fixture using other fixtures"""
    user = User.create("test@example.com")
    db_connection.save(user)
    return user

@pytest.fixture
def authenticated_client(user):
    """Create authenticated client"""
    client = Client()
    client.login(user)
    return client

def test_user_profile(authenticated_client):
    """Test with composite fixture"""
    profile = authenticated_client.get_profile()
    assert profile.email == "test@example.com"

# Parametrized fixtures
@pytest.fixture(params=["admin", "user", "guest"])
def user_role(request):
    """Fixture that runs for each parameter"""
    role = request.param
    user = User(role=role)
    yield user
    print(f"Cleaning up {role} user")

def test_access_control(user_role):
    """Test runs 3 times with different roles"""
    assert user_role.role in ["admin", "user", "guest"]

# Factory fixtures
@pytest.fixture
def make_user():
    """Factory fixture that returns a function"""
    created_users = []
    
    def _make_user(email, role="user"):
        user = User(email, role)
        created_users.append(user)
        return user
    
    yield _make_user
    
    # Cleanup all created users
    for user in created_users:
        user.delete()

def test_multiple_users(make_user):
    """Create users dynamically"""
    admin = make_user("admin@test.com", "admin")
    user1 = make_user("user1@test.com")
    user2 = make_user("user2@test.com")
    
    assert admin.role == "admin"
    assert user1.role == "user"
    assert user2.role == "user"

# Temporary directories
def test_file_operations(tmp_path):
    """Use pytest's built-in tmp_path fixture"""
    data_file = tmp_path / "data.txt"
    data_file.write_text("Hello, World!")
    
    assert data_file.read_text() == "Hello, World!"
    assert data_file.exists()
    
    # Cleanup automatically handled

# Monkeypatch (mocking)
def test_env_variable(monkeypatch):
    """Mock environment variables"""
    monkeypatch.setenv("API_KEY", "test-key-123")
    assert os.environ["API_KEY"] == "test-key-123"

def test_object_attribute(monkeypatch):
    """Mock object attributes"""
    calculator = Calculator()
    monkeypatch.setattr(calculator, "mode", "scientific")
    assert calculator.mode == "scientific"

def test_function(monkeypatch):
    """Mock function/method"""
    def mock_add(a, b):
        return 42
    
    monkeypatch.setattr("module_to_test.add", mock_add)
    assert add(1, 2) == 42

def test_dictionary(monkeypatch):
    """Mock dict items"""
    config = {"debug": False}
    monkeypatch.setitem(config, "debug", True)
    assert config["debug"] is True

# Testing exceptions with context
def test_exception_with_cleanup():
    """Test exception while ensuring cleanup"""
    cleanup_called = False
    
    def cleanup():
        nonlocal cleanup_called
        cleanup_called = True
    
    with pytest.raises(ValueError):
        try:
            raise ValueError("Error")
        finally:
            cleanup()
    
    assert cleanup_called

# Async tests (pytest-asyncio)
@pytest.mark.asyncio
async def test_async_function():
    """Test async function"""
    result = await async_add(2, 3)
    assert result == 5

# Custom assertion messages
def test_with_custom_message():
    """Add custom assertion messages"""
    actual = [1, 2, 3]
    expected = [1, 2, 4]
    
    assert actual == expected, f"Expected {expected}, but got {actual}"

# Test skipping with dynamic reason
def test_dynamic_skip():
    """Skip based on runtime condition"""
    if not is_feature_available():
        pytest.skip("Feature not available")
    assert feature_function() == expected

# Test failure with custom message
def test_custom_failure():
    """Force test failure with message"""
    if not can_continue():
        pytest.fail("Cannot continue test")

# Helper functions
def add(a: int, b: int) -> int:
    return a + b

def subtract(a: int, b: int) -> int:
    return a - b

class Calculator:
    def __init__(self):
        self.mode = "basic"
    
    def add(self, a, b):
        return a + b
    
    def subtract(self, a, b):
        return a - b
    
    def multiply(self, a, b):
        return a * b
    
    def divide(self, a, b):
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    
    def calculate(self, a, b):
        return {
            'sum': self.add(a, b),
            'difference': self.subtract(a, b),
            'product': self.multiply(a, b),
            'quotient': self.divide(a, b)
        }
    
    def clear(self):
        pass

async def async_add(a, b):
    return a + b

def is_feature_available():
    return False

def feature_function():
    return 42

def can_continue():
    return False

class User:
    def __init__(self, email=None, role="user"):
        self.email = email
        self.role = role
    
    def delete(self):
        pass

class Client:
    def login(self, user):
        pass
    
    def get_profile(self):
        return type('obj', (object,), {'email': 'test@example.com'})()

class DatabaseConnection:
    def __init__(self, db_name):
        self.db_name = db_name
    
    def save(self, obj):
        pass
    
    def close(self):
        pass

def User_create(email):
    return User(email)
```

### **42.2.3 Mocking with unittest.mock**

```python
# test_user_service.py
from unittest.mock import Mock, patch, MagicMock, PropertyMock, call
import pytest
import requests
from datetime import datetime

# Basic mocking
def test_mock_basics():
    """Demonstrate basic mocking concepts"""
    # Create mock
    mock = Mock()
    
    # Configure return value
    mock.get_data.return_value = {"id": 1, "name": "Test"}
    
    # Call mock
    result = mock.get_data(42)
    
    # Assertions
    assert result == {"id": 1, "name": "Test"}
    mock.get_data.assert_called_once_with(42)
    mock.get_data.assert_called_with(42)
    assert mock.get_data.call_count == 1

# Mock with side effects
def test_mock_side_effects():
    """Different ways to use side_effect"""
    mock = Mock()
    
    # Side effect as exception
    mock.fetch.side_effect = ValueError("API Error")
    
    with pytest.raises(ValueError):
        mock.fetch()
    
    # Side effect as iterable
    mock.get_values.side_effect = [1, 2, 3, StopIteration]
    assert mock.get_values() == 1
    assert mock.get_values() == 2
    assert mock.get_values() == 3
    with pytest.raises(StopIteration):
        mock.get_values()
    
    # Side effect as function
    def side_effect_func(*args, **kwargs):
        return f"Processed {args} {kwargs}"
    
    mock.process.side_effect = side_effect_func
    assert mock.process("test", flag=True) == "Processed ('test',) {'flag': True}"

# Mocking classes and instances
@patch('module.ClassName')
def test_mock_class(mock_class):
    """Mock entire class"""
    # Configure mock instance
    instance = mock_class.return_value
    instance.method.return_value = "mocked"
    
    # Call code that uses the class
    result = use_class()
    
    assert result == "mocked"
    mock_class.assert_called_once()
    instance.method.assert_called_once()

# Patch decorator with context
def test_patch_context():
    """Use patch as context manager"""
    with patch('requests.get') as mock_get:
        # Configure mock
        mock_response = Mock()
        mock_response.status_code = 200
        mock_response.json.return_value = {"data": "test"}
        mock_get.return_value = mock_response
        
        # Call function that uses requests.get
        result = fetch_data("https://api.example.com")
        
        assert result == {"data": "test"}
        mock_get.assert_called_once_with("https://api.example.com")

# Multiple patches
@patch('module.db.save')
@patch('module.cache.set')
@patch('module.logger.info')
def test_multiple_patches(mock_log, mock_cache, mock_db):
    """Test with multiple mocks (note order: bottom to top)"""
    process_data({"id": 1})
    
    mock_db.assert_called_once()
    mock_cache.assert_called_once()
    mock_log.assert_called_once()

# Mock properties
class TestProperties:
    def test_property_mocking(self):
        """Mock class properties"""
        mock_obj = Mock()
        
        # Mock property
        p = PropertyMock(return_value="mocked_property")
        type(mock_obj).property_name = p
        
        # Access property
        result = mock_obj.property_name
        
        assert result == "mocked_property"
        p.assert_called_once()

# Real-world example: UserService testing
class TestUserService:
    
    @pytest.fixture
    def user_service(self):
        """Create service with mocked dependencies"""
        self.mock_db = Mock(spec=Database)  # spec ensures only valid methods
        self.mock_email = Mock(spec=EmailService)
        self.mock_logger = Mock(spec=Logger)
        
        service = UserService(
            db=self.mock_db,
            email=self.mock_email,
            logger=self.mock_logger
        )
        return service
    
    def test_create_user_success(self, user_service):
        # Arrange
        email = "test@example.com"
        name = "Test User"
        
        self.mock_db.find_user.return_value = None
        self.mock_db.save_user.return_value = User(id=1, email=email, name=name)
        
        # Act
        result = user_service.create_user(email, name)
        
        # Assert
        assert result.id == 1
        assert result.email == email
        
        # Verify calls
        self.mock_db.find_user.assert_called_once_with(email)
        self.mock_db.save_user.assert_called_once()
        self.mock_email.send_welcome.assert_called_once_with(email, name)
        self.mock_logger.info.assert_called()
    
    def test_create_user_already_exists(self, user_service):
        # Arrange
        email = "existing@example.com"
        existing_user = User(id=1, email=email, name="Existing")
        self.mock_db.find_user.return_value = existing_user
        
        # Act & Assert
        with pytest.raises(UserExistsError):
            user_service.create_user(email, "New User")
        
        # Verify save was NOT called
        self.mock_db.save_user.assert_not_called()
        self.mock_email.send_welcome.assert_not_called()
    
    def test_get_user_by_id(self, user_service):
        # Arrange
        user_id = 42
        expected_user = User(id=user_id, email="test@example.com")
        self.mock_db.get_user.return_value = expected_user
        
        # Act
        result = user_service.get_user(user_id)
        
        # Assert
        assert result == expected_user
        self.mock_db.get_user.assert_called_once_with(user_id)
    
    def test_get_user_not_found(self, user_service):
        # Arrange
        self.mock_db.get_user.return_value = None
        
        # Act & Assert
        with pytest.raises(UserNotFoundError):
            user_service.get_user(999)
    
    def test_update_user(self, user_service):
        # Arrange
        user_id = 1
        updates = {"name": "New Name", "email": "new@example.com"}
        
        existing_user = User(id=user_id, email="old@example.com", name="Old Name")
        self.mock_db.get_user.return_value = existing_user
        
        # Act
        user_service.update_user(user_id, updates)
        
        # Assert
        assert existing_user.name == "New Name"
        assert existing_user.email == "new@example.com"
        self.mock_db.save_user.assert_called_once_with(existing_user)
    
    def test_update_user_partial(self, user_service):
        # Arrange
        user_id = 1
        updates = {"name": "New Name Only"}
        
        existing_user = User(id=user_id, email="test@example.com", name="Old Name")
        self.mock_db.get_user.return_value = existing_user
        
        # Act
        user_service.update_user(user_id, updates)
        
        # Assert
        assert existing_user.name == "New Name Only"
        assert existing_user.email == "test@example.com"  # Unchanged
        self.mock_db.save_user.assert_called_once()
    
    def test_get_all_users(self, user_service):
        # Arrange
        expected_users = [
            User(id=1, email="user1@example.com"),
            User(id=2, email="user2@example.com")
        ]
        self.mock_db.get_all_users.return_value = expected_users
        
        # Act
        result = user_service.get_all_users()
        
        # Assert
        assert result == expected_users
        assert len(result) == 2
    
    def test_delete_user(self, user_service):
        # Arrange
        user_id = 1
        user = User(id=user_id)
        self.mock_db.get_user.return_value = user
        
        # Act
        user_service.delete_user(user_id)
        
        # Assert
        self.mock_db.delete_user.assert_called_once_with(user)
        self.mock_logger.info.assert_called_with(f"Deleted user {user_id}")
    
    def test_bulk_create(self, user_service):
        # Arrange
        user_data = [
            {"email": "user1@example.com", "name": "User 1"},
            {"email": "user2@example.com", "name": "User 2"}
        ]
        
        self.mock_db.find_user.return_value = None
        self.mock_db.save_user.side_effect = [
            User(id=1, **user_data[0]),
            User(id=2, **user_data[1])
        ]
        
        # Act
        results = user_service.bulk_create(user_data)
        
        # Assert
        assert len(results) == 2
        assert self.mock_db.save_user.call_count == 2
        assert self.mock_email.send_welcome.call_count == 2
    
    def test_search_users(self, user_service):
        # Arrange
        criteria = {"role": "admin"}
        expected = [User(id=1, role="admin"), User(id=2, role="admin")]
        self.mock_db.search_users.return_value = expected
        
        # Act
        results = user_service.search_users(criteria)
        
        # Assert
        assert results == expected
        self.mock_db.search_users.assert_called_once_with(criteria)
    
    def test_logger_called_with_correct_level(self, user_service):
        # Arrange
        self.mock_db.find_user.return_value = None
        self.mock_db.save_user.return_value = User(id=1, email="test@example.com")
        
        # Act
        user_service.create_user("test@example.com", "Test")
        
        # Assert logger calls with correct levels
        self.mock_logger.info.assert_has_calls([
            call("Creating user with email: test@example.com"),
            call("User created with ID: 1")
        ])
        
        # Verify no error logs
        self.mock_logger.error.assert_not_called()

# Mocking external APIs
@patch('requests.get')
def test_external_api(mock_get):
    """Test code that calls external API"""
    # Mock response
    mock_response = Mock()
    mock_response.status_code = 200
    mock_response.json.return_value = {
        "users": [
            {"id": 1, "name": "Alice"},
            {"id": 2, "name": "Bob"}
        ]
    }
    mock_get.return_value = mock_response
    
    # Call function
    users = fetch_users_from_api()
    
    # Assert
    assert len(users) == 2
    assert users[0]["name"] == "Alice"
    mock_get.assert_called_once_with("https://api.example.com/users")

@patch('requests.post')
def test_api_with_authentication(mock_post):
    """Test API with authentication"""
    # Mock successful authentication
    mock_response = Mock()
    mock_response.status_code = 200
    mock_response.json.return_value = {"token": "jwt-token-123"}
    mock_post.return_value = mock_response
    
    # Call function
    token = authenticate("user", "pass")
    
    # Assert correct call
    mock_post.assert_called_once_with(
        "https://api.example.com/auth",
        json={"username": "user", "password": "pass"},
        headers={"Content-Type": "application/json"},
        timeout=30
    )
    assert token == "jwt-token-123"

# Mock datetime
def test_datetime_mocking():
    """Mock datetime to control time in tests"""
    with patch('datetime.datetime') as mock_datetime:
        # Configure mock to return fixed datetime
        fixed_date = datetime(2024, 1, 15, 10, 30, 0)
        mock_datetime.now.return_value = fixed_date
        
        # Call function that uses datetime.now()
        result = get_current_timestamp()
        
        assert result == "2024-01-15 10:30:00"

# Mock environment variables
def test_environment_variables():
    """Mock os.environ for testing"""
    with patch.dict('os.environ', {'API_KEY': 'test-key', 'DEBUG': 'true'}):
        assert os.environ['API_KEY'] == 'test-key'
        assert os.environ['DEBUG'] == 'true'
    
    # Original environment restored after context

# Mock file operations
def test_file_reading():
    """Mock file operations"""
    mock_open = mock_open(read_data="file content")
    
    with patch('builtins.open', mock_open):
        with open('file.txt', 'r') as f:
            content = f.read()
        
        assert content == "file content"
        
        # Verify correct file was opened
        mock_open.assert_called_once_with('file.txt', 'r')

# Supporting classes and functions
class Database:
    def find_user(self, email):
        pass
    
    def save_user(self, user):
        pass
    
    def get_user(self, user_id):
        pass
    
    def delete_user(self, user):
        pass
    
    def get_all_users(self):
        pass
    
    def search_users(self, criteria):
        pass

class EmailService:
    def send_welcome(self, email, name):
        pass

class Logger:
    def info(self, msg):
        pass
    
    def error(self, msg):
        pass
    
    def debug(self, msg):
        pass

class User:
    def __init__(self, id=None, email=None, name=None, role="user"):
        self.id = id
        self.email = email
        self.name = name
        self.role = role

class UserExistsError(Exception):
    pass

class UserNotFoundError(Exception):
    pass

class UserService:
    def __init__(self, db, email, logger):
        self.db = db
        self.email = email
        self.logger = logger
    
    def create_user(self, email, name):
        self.logger.info(f"Creating user with email: {email}")
        
        if self.db.find_user(email):
            raise UserExistsError(f"User {email} already exists")
        
        user = User(email=email, name=name)
        saved = self.db.save_user(user)
        self.email.send_welcome(email, name)
        self.logger.info(f"User created with ID: {saved.id}")
        return saved
    
    def get_user(self, user_id):
        user = self.db.get_user(user_id)
        if not user:
            raise UserNotFoundError(f"User {user_id} not found")
        return user
    
    def update_user(self, user_id, updates):
        user = self.get_user(user_id)
        for key, value in updates.items():
            if hasattr(user, key):
                setattr(user, key, value)
        self.db.save_user(user)
    
    def delete_user(self, user_id):
        user = self.get_user(user_id)
        self.db.delete_user(user)
        self.logger.info(f"Deleted user {user_id}")
    
    def get_all_users(self):
        return self.db.get_all_users()
    
    def bulk_create(self, user_data_list):
        results = []
        for data in user_data_list:
            user = self.create_user(data["email"], data["name"])
            results.append(user)
        return results
    
    def search_users(self, criteria):
        return self.db.search_users(criteria)

def use_class():
    from module import ClassName
    instance = ClassName()
    return instance.method()

def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def process_data(data):
    from module import db, cache, logger
    db.save(data)
    cache.set(data['id'], data)
    logger.info("Data processed")

def fetch_users_from_api():
    response = requests.get("https://api.example.com/users")
    return response.json().get("users", [])

def authenticate(username, password):
    response = requests.post(
        "https://api.example.com/auth",
        json={"username": username, "password": password},
        headers={"Content-Type": "application/json"},
        timeout=30
    )
    return response.json().get("token")

def get_current_timestamp():
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
```

---

## **42.3 Jest (JavaScript)**

### **42.3.1 Introduction to Jest**

Jest is a delightful JavaScript testing framework with a focus on simplicity. It works with projects using Babel, TypeScript, Node.js, React, Angular, Vue.js, and provides built-in assertions, mocking, and code coverage.

### **42.3.2 Basic Test Structure**

```javascript
// calculator.test.js
const { Calculator, UserService, fetchData, processAsync } = require('./app');

// Basic test
test('adds 1 + 2 to equal 3', () => {
    const calculator = new Calculator();
    expect(calculator.add(1, 2)).toBe(3);
});

// Using describe to group tests
describe('Calculator', () => {
    let calculator;
    
    // Setup before each test
    beforeEach(() => {
        calculator = new Calculator();
    });
    
    // Cleanup after each test
    afterEach(() => {
        calculator = null;
    });
    
    // Setup once before all tests
    beforeAll(() => {
        console.log('Starting calculator tests');
    });
    
    // Cleanup once after all tests
    afterAll(() => {
        console.log('Finished calculator tests');
    });
    
    test('addition', () => {
        expect(calculator.add(2, 3)).toBe(5);
        expect(calculator.add(-1, 1)).toBe(0);
    });
    
    test('subtraction', () => {
        expect(calculator.subtract(5, 3)).toBe(2);
        expect(calculator.subtract(3, 5)).toBe(-2);
    });
    
    test('multiplication', () => {
        expect(calculator.multiply(4, 3)).toBe(12);
        expect(calculator.multiply(0, 5)).toBe(0);
    });
    
    test('division', () => {
        expect(calculator.divide(10, 2)).toBe(5);
        expect(calculator.divide(7, 2)).toBeCloseTo(3.5);
    });
    
    test('division by zero throws error', () => {
        expect(() => {
            calculator.divide(10, 0);
        }).toThrow('Cannot divide by zero');
    });
    
    // Nested describe blocks
    describe('with negative numbers', () => {
        test('addition', () => {
            expect(calculator.add(-2, -3)).toBe(-5);
        });
        
        test('multiplication', () => {
            expect(calculator.multiply(-2, 3)).toBe(-6);
            expect(calculator.multiply(-2, -3)).toBe(6);
        });
    });
});

// Different matchers
describe('Matchers', () => {
    test('equality', () => {
        expect(2 + 2).toBe(4);
        expect({name: 'John'}).toEqual({name: 'John'});
        expect([1, 2, 3]).toEqual([1, 2, 3]);
    });
    
    test('truthiness', () => {
        expect(true).toBeTruthy();
        expect(false).toBeFalsy();
        expect(null).toBeNull();
        expect(undefined).toBeUndefined();
        expect('hello').toBeDefined();
    });
    
    test('numbers', () => {
        expect(10).toBeGreaterThan(5);
        expect(10).toBeGreaterThanOrEqual(10);
        expect(5).toBeLessThan(10);
        expect(5).toBeLessThanOrEqual(5);
        
        // Floating point
        expect(0.1 + 0.2).toBeCloseTo(0.3);
    });
    
    test('strings', () => {
        expect('Hello World').toMatch(/World/);
        expect('Hello World').toContain('World');
        expect('Hello World').toHaveLength(11);
    });
    
    test('arrays', () => {
        const shoppingList = ['milk', 'eggs', 'bread'];
        
        expect(shoppingList).toContain('eggs');
        expect(shoppingList).toHaveLength(3);
        expect(shoppingList).toEqual(expect.arrayContaining(['milk', 'bread']));
    });
    
    test('objects', () => {
        const user = {
            name: 'John',
            age: 30,
            address: {
                city: 'New York',
                country: 'USA'
            }
        };
        
        expect(user).toHaveProperty('name');
        expect(user).toHaveProperty('age', 30);
        expect(user).toMatchObject({name: 'John'});
        expect(user).toEqual({
            name: 'John',
            age: 30,
            address: expect.objectContaining({city: 'New York'})
        });
    });
});

// Testing asynchronous code
describe('Async Testing', () => {
    // Callbacks
    test('callback-based', (done) => {
        function callback(data) {
            try {
                expect(data).toBe('peanut butter');
                done();
            } catch (error) {
                done(error);
            }
        }
        
        fetchData(callback);
    });
    
    // Promises - return promise
    test('promise-based', () => {
        return fetchData().then(data => {
            expect(data).toBe('peanut butter');
        });
    });
    
    // Promises - resolves/rejects matchers
    test('resolves matcher', () => {
        return expect(fetchData()).resolves.toBe('peanut butter');
    });
    
    test('rejects matcher', () => {
        return expect(fetchData(true)).rejects.toThrow('Error');
    });
    
    // Async/await
    test('async/await', async () => {
        const data = await fetchData();
        expect(data).toBe('peanut butter');
    });
    
    test('async/await with error', async () => {
        await expect(fetchData(true)).rejects.toThrow('Error');
    });
});

// Parameterized tests
describe('Parameterized Tests', () => {
    test.each([
        [1, 1, 2],
        [2, 3, 5],
        [5, -2, 3],
        [0, 0, 0]
    ])('adds %i + %i = %i', (a, b, expected) => {
        const calculator = new Calculator();
        expect(calculator.add(a, b)).toBe(expected);
    });
    
    test.each([
        {a: 2, b: 3, expected: 6},
        {a: 4, b: 5, expected: 20},
        {a: 10, b: 0, expected: 0}
    ])('multiplies $a * $b = $expected', ({a, b, expected}) => {
        const calculator = new Calculator();
        expect(calculator.multiply(a, b)).toBe(expected);
    });
    
    // Table with template literal
    test.each`
        a    | b    | expected
        ${1} | ${2} | ${3}
        ${3} | ${4} | ${7}
        ${5} | ${6} | ${11}
    `('adds $a + $b = $expected', ({a, b, expected}) => {
        const calculator = new Calculator();
        expect(calculator.add(a, b)).toBe(expected);
    });
});

// Skipping and focusing tests
describe('Test Selection', () => {
    test.skip('this test will be skipped', () => {
        expect(true).toBe(false);
    });
    
    test.only('this test will run exclusively', () => {
        expect(true).toBe(true);
    });
    
    // Use .only on describe block to run only that block
    describe.only('Focused group', () => {
        test('test in focused group', () => {
            expect(true).toBe(true);
        });
    });
    
    // Conditional skipping
    const isFeatureEnabled = false;
    (isFeatureEnabled ? test : test.skip)('conditional skip', () => {
        // Test implementation
    });
});

// Testing timeouts
describe('Timeouts', () => {
    test('respects timeout', async () => {
        // Default timeout is 5000ms
        const result = await slowOperation();
        expect(result).toBe('done');
    }, 10000); // Custom timeout of 10 seconds
    
    test('fails on timeout', async () => {
        jest.setTimeout(1000); // Set timeout for this test
        await expect(slowOperation(2000)).rejects.toThrow('Timeout');
    });
});

// Mock functions
describe('Mock Functions', () => {
    test('mock implementation', () => {
        const mock = jest.fn(x => x * 2);
        
        expect(mock(5)).toBe(10);
        expect(mock(3)).toBe(6);
        expect(mock).toHaveBeenCalledWith(5);
        expect(mock).toHaveBeenCalledWith(3);
        expect(mock).toHaveBeenCalledTimes(2);
    });
    
    test('mock return values', () => {
        const mock = jest.fn();
        
        mock.mockReturnValueOnce(10).mockReturnValueOnce(20).mockReturnValue(30);
        
        expect(mock()).toBe(10);
        expect(mock()).toBe(20);
        expect(mock()).toBe(30);
        expect(mock()).toBe(30);
    });
    
    test('mock resolved promises', async () => {
        const mock = jest.fn();
        mock.mockResolvedValue('success');
        
        await expect(mock()).resolves.toBe('success');
        
        mock.mockRejectedValue(new Error('failure'));
        await expect(mock()).rejects.toThrow('failure');
    });
    
    test('mock implementation once', () => {
        const mock = jest.fn()
            .mockImplementationOnce(() => 'first call')
            .mockImplementationOnce(() => 'second call')
            .mockImplementation(() => 'default');
        
        expect(mock()).toBe('first call');
        expect(mock()).toBe('second call');
        expect(mock()).toBe('default');
        expect(mock()).toBe('default');
    });
    
    test('inspect mock calls', () => {
        const mock = jest.fn();
        
        mock('arg1', 'arg2');
        mock('arg3');
        
        expect(mock.mock.calls).toEqual([
            ['arg1', 'arg2'],
            ['arg3']
        ]);
        
        expect(mock.mock.calls[0][0]).toBe('arg1');
        expect(mock.mock.calls[0][1]).toBe('arg2');
        expect(mock.mock.calls[1][0]).toBe('arg3');
        
        expect(mock.mock.results[0].value).toBeUndefined();
    });
});

// Module mocking
jest.mock('./api');
const api = require('./api');

describe('Module Mocking', () => {
    beforeEach(() => {
        jest.clearAllMocks();
    });
    
    test('mock entire module', async () => {
        // Configure mock
        api.getUsers.mockResolvedValue([
            {id: 1, name: 'John'},
            {id: 2, name: 'Jane'}
        ]);
        
        // Call function that uses the module
        const users = await fetchUsers();
        
        expect(users).toHaveLength(2);
        expect(api.getUsers).toHaveBeenCalledTimes(1);
        expect(api.getUsers).toHaveBeenCalledWith({active: true});
    });
    
    test('mock specific function', () => {
        const mockSave = jest.fn().mockReturnValue(true);
        api.saveUser.mockImplementation(mockSave);
        
        const result = saveUser({name: 'Test'});
        
        expect(result).toBe(true);
        expect(api.saveUser).toHaveBeenCalledWith({name: 'Test'});
    });
    
    test('mock with different responses', async () => {
        api.getUser
            .mockResolvedValueOnce({id: 1, name: 'First'})
            .mockResolvedValueOnce({id: 2, name: 'Second'})
            .mockResolvedValue(null);
        
        const user1 = await api.getUser(1);
        const user2 = await api.getUser(2);
        const user3 = await api.getUser(3);
        
        expect(user1.name).toBe('First');
        expect(user2.name).toBe('Second');
        expect(user3).toBeNull();
    });
});

// Partial mocking with jest.requireActual
jest.mock('./config', () => ({
    ...jest.requireActual('./config'),
    getApiKey: jest.fn(() => 'mock-key')
}));

// Mocking classes
jest.mock('./database');

describe('Class Mocking', () => {
    test('mock class constructor', () => {
        const Database = require('./database');
        
        // Mock instance methods
        const mockConnect = jest.fn();
        const mockQuery = jest.fn().mockResolvedValue([{id: 1}]);
        
        Database.mockImplementation(() => ({
            connect: mockConnect,
            query: mockQuery
        }));
        
        const db = new Database('test');
        db.connect();
        db.query('SELECT * FROM users');
        
        expect(mockConnect).toHaveBeenCalled();
        expect(mockQuery).toHaveBeenCalledWith('SELECT * FROM users');
    });
    
    test('mock class with mockImplementation', () => {
        const Database = require('./database');
        
        const mockDb = {
            connect: jest.fn(),
            disconnect: jest.fn(),
            find: jest.fn().mockReturnValue([])
        };
        
        Database.mockImplementation(() => mockDb);
        
        const db = new Database();
        db.connect();
        db.find({name: 'John'});
        
        expect(db.connect).toHaveBeenCalled();
        expect(db.find).toHaveBeenCalledWith({name: 'John'});
    });
});

// Timer mocks
jest.useFakeTimers();

describe('Timer Mocking', () => {
    afterEach(() => {
        jest.useRealTimers();
    });
    
    test('setTimeout', () => {
        const callback = jest.fn();
        
        setTimeout(callback, 1000);
        
        // Fast-forward time
        jest.advanceTimersByTime(1000);
        
        expect(callback).toHaveBeenCalled();
        expect(callback).toHaveBeenCalledTimes(1);
    });
    
    test('runAllTimers', () => {
        const callback = jest.fn();
        
        setTimeout(callback, 1000);
        setTimeout(callback, 2000);
        
        jest.runAllTimers();
        
        expect(callback).toHaveBeenCalledTimes(2);
    });
    
    test('runOnlyPendingTimers', () => {
        const callback1 = jest.fn();
        const callback2 = jest.fn();
        
        setTimeout(callback1, 1000);
        setTimeout(callback2, 2000);
        
        jest.advanceTimersByTime(1500);
        
        expect(callback1).toHaveBeenCalled();
        expect(callback2).not.toHaveBeenCalled();
    });
    
    test('clearTimeout', () => {
        const callback = jest.fn();
        
        const timerId = setTimeout(callback, 1000);
        clearTimeout(timerId);
        
        jest.runAllTimers();
        
        expect(callback).not.toHaveBeenCalled();
    });
    
    test('setInterval', () => {
        const callback = jest.fn();
        
        setInterval(callback, 1000);
        
        jest.advanceTimersByTime(3000);
        
        expect(callback).toHaveBeenCalledTimes(3);
    });
});

// Snapshot testing
describe('Snapshot Testing', () => {
    test('user object snapshot', () => {
        const user = {
            id: 1,
            name: 'John Doe',
            email: 'john@example.com',
            createdAt: new Date('2024-01-15')
        };
        
        expect(user).toMatchSnapshot();
    });
    
    test('dynamic content snapshot with property matchers', () => {
        const user = {
            id: Math.floor(Math.random() * 1000),
            name: 'Jane Doe',
            createdAt: new Date()
        };
        
        expect(user).toMatchSnapshot({
            id: expect.any(Number),
            createdAt: expect.any(Date)
        });
    });
    
    test('inline snapshot', () => {
        const data = processData('test');
        expect(data).toMatchInlineSnapshot(`"PROCESSED: TEST"`);
    });
});

// Testing errors
describe('Error Testing', () => {
    test('throws specific error', () => {
        expect(() => {
            throw new Error('Invalid input');
        }).toThrow('Invalid input');
        
        expect(() => {
            throw new TypeError('Type error');
        }).toThrow(TypeError);
    });
    
    test('error message matches regex', () => {
        expect(() => {
            throw new Error('Invalid input: email');
        }).toThrow(/invalid.*email/i);
    });
});

// Real-world example: testing React components
describe('React Component Testing', () => {
    // Assuming React and Testing Library are set up
    
    test('renders user name', () => {
        // const { getByText } = render(<UserProfile user={{name: 'John'}} />);
        // expect(getByText('John')).toBeInTheDocument();
    });
    
    test('calls onClick handler', () => {
        const handleClick = jest.fn();
        
        // const { getByRole } = render(<Button onClick={handleClick}>Click me</Button>);
        // fireEvent.click(getByRole('button'));
        
        expect(handleClick).toHaveBeenCalledTimes(1);
    });
    
    test('displays loading state', () => {
        // const { getByTestId } = render(<DataLoader loading={true} />);
        // expect(getByTestId('loading-spinner')).toBeInTheDocument();
    });
});

// Integration test example
describe('UserService Integration', () => {
    let userService;
    let mockDb;
    let mockEmail;
    
    beforeEach(() => {
        mockDb = {
            findUser: jest.fn(),
            saveUser: jest.fn(),
            getUser: jest.fn()
        };
        
        mockEmail = {
            sendWelcome: jest.fn()
        };
        
        userService = new UserService(mockDb, mockEmail);
    });
    
    test('create user successfully', async () => {
        // Arrange
        const userData = {
            email: 'test@example.com',
            name: 'Test User'
        };
        
        mockDb.findUser.mockResolvedValue(null);
        mockDb.saveUser.mockImplementation(user => ({
            ...user,
            id: 1
        }));
        
        // Act
        const result = await userService.createUser(userData.email, userData.name);
        
        // Assert
        expect(result).toEqual({
            id: 1,
            email: 'test@example.com',
            name: 'Test User'
        });
        
        expect(mockDb.findUser).toHaveBeenCalledWith('test@example.com');
        expect(mockDb.saveUser).toHaveBeenCalled();
        expect(mockEmail.sendWelcome).toHaveBeenCalledWith('test@example.com', 'Test User');
    });
    
    test('create user - already exists', async () => {
        // Arrange
        const existingUser = {id: 1, email: 'test@example.com'};
        mockDb.findUser.mockResolvedValue(existingUser);
        
        // Act & Assert
        await expect(userService.createUser('test@example.com', 'Test'))
            .rejects.toThrow('User already exists');
        
        expect(mockDb.saveUser).not.toHaveBeenCalled();
        expect(mockEmail.sendWelcome).not.toHaveBeenCalled();
    });
});

// Helper functions and classes
class Calculator {
    add(a, b) { return a + b; }
    subtract(a, b) { return a - b; }
    multiply(a, b) { return a * b; }
    divide(a, b) {
        if (b === 0) throw new Error('Cannot divide by zero');
        return a / b;
    }
}

function fetchData(shouldFail = false) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            if (shouldFail) {
                reject(new Error('Error'));
            } else {
                resolve('peanut butter');
            }
        }, 100);
    });
}

function slowOperation(delay = 1000) {
    return new Promise(resolve => {
        setTimeout(() => resolve('done'), delay);
    });
}

class UserService {
    constructor(db, email) {
        this.db = db;
        this.email = email;
    }
    
    async createUser(email, name) {
        const existing = await this.db.findUser(email);
        if (existing) {
            throw new Error('User already exists');
        }
        
        const user = {email, name};
        const saved = await this.db.saveUser(user);
        await this.email.sendWelcome(email, name);
        return saved;
    }
}

// Mocked modules
const api = {
    getUsers: jest.fn(),
    getUser: jest.fn(),
    saveUser: jest.fn()
};

class Database {
    constructor(connection) {}
    connect() {}
    query(sql) {}
    disconnect() {}
}

// Functions using mocks
async function fetchUsers() {
    return api.getUsers({active: true});
}

function saveUser(user) {
    return api.saveUser(user);
}

function processData(input) {
    return `PROCESSED: ${input.toUpperCase()}`;
}
```

### **42.3.3 Advanced Jest Features**

```javascript
// jest.config.js - Jest configuration
module.exports = {
    // Test environment
    testEnvironment: 'node',  // or 'jsdom' for browser
    
    // Test file patterns
    testMatch: [
        '**/__tests__/**/*.js',
        '**/?(*.)+(spec|test).js'
    ],
    
    // Coverage collection
    collectCoverage: true,
    collectCoverageFrom: [
        'src/**/*.js',
        '!src/**/*.test.js',
        '!src/index.js'
    ],
    coverageThreshold: {
        global: {
            branches: 80,
            functions: 80,
            lines: 80,
            statements: 80
        }
    },
    
    // Module aliases
    moduleNameMapper: {
        '^@/(.*)$': '<rootDir>/src/$1'
    },
    
    // Setup files
    setupFilesAfterEnv: ['<rootDir>/jest.setup.js'],
    
    // Transform for modern JS
    transform: {
        '^.+\\.js$': 'babel-jest'
    },
    
    // Test timeout
    testTimeout: 10000,
    
    // Verbose output
    verbose: true
};

// jest.setup.js - Global setup
beforeAll(() => {
    console.log('Global setup');
});

afterAll(() => {
    console.log('Global teardown');
});

// Custom matchers
expect.extend({
    toBeWithinRange(received, floor, ceiling) {
        const pass = received >= floor && received <= ceiling;
        if (pass) {
            return {
                message: () =>
                    `expected ${received} not to be within range ${floor} - ${ceiling}`,
                pass: true
            };
        } else {
            return {
                message: () =>
                    `expected ${received} to be within range ${floor} - ${ceiling}`,
                pass: false
            };
        }
    },
    
    toBeValidEmail(received) {
        const emailRegex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
        const pass = emailRegex.test(received);
        
        return {
            message: () =>
                `expected ${received} to be a valid email address`,
            pass
        };
    }
});

// Using custom matchers
describe('Custom Matchers', () => {
    test('numbers within range', () => {
        expect(5).toBeWithinRange(1, 10);
        expect(15).not.toBeWithinRange(1, 10);
    });
    
    test('valid email', () => {
        expect('test@example.com').toBeValidEmail();
        expect('invalid-email').not.toBeValidEmail();
    });
});

// Test hooks with descriptions
describe('Hooks Order', () => {
    beforeAll(() => console.log('1 - beforeAll'));
    afterAll(() => console.log('6 - afterAll'));
    beforeEach(() => console.log('2 - beforeEach'));
    afterEach(() => console.log('5 - afterEach'));
    
    test('test 1', () => console.log('3 - test 1'));
    test('test 2', () => console.log('4 - test 2'));
});

// Testing with different environments
describe('Environment Specific', () => {
    const originalEnv = process.env.NODE_ENV;
    
    beforeEach(() => {
        jest.resetModules();
    });
    
    afterEach(() => {
        process.env.NODE_ENV = originalEnv;
    });
    
    test('development behavior', () => {
        process.env.NODE_ENV = 'development';
        const config = require('./config');
        expect(config.apiUrl).toBe('http://localhost:3000');
    });
    
    test('production behavior', () => {
        process.env.NODE_ENV = 'production';
        const config = require('./config');
        expect(config.apiUrl).toBe('https://api.example.com');
    });
});

// Testing with spies
describe('Spies', () => {
    const obj = {
        method: (x) => x * 2,
        asyncMethod: async (x) => x * 2
    };
    
    test('spy on method', () => {
        const spy = jest.spyOn(obj, 'method');
        
        const result = obj.method(5);
        
        expect(result).toBe(10);
        expect(spy).toHaveBeenCalledWith(5);
        expect(spy).toHaveBeenCalledTimes(1);
        
        spy.mockRestore();
    });
    
    test('spy and mock implementation', () => {
        const spy = jest.spyOn(obj, 'method').mockImplementation(x => x * 3);
        
        const result = obj.method(5);
        
        expect(result).toBe(15);
        expect(spy).toHaveBeenCalledWith(5);
        
        spy.mockRestore();
    });
    
    test('spy on async method', async () => {
        const spy = jest.spyOn(obj, 'asyncMethod');
        
        const result = await obj.asyncMethod(5);
        
        expect(result).toBe(10);
        expect(spy).toHaveBeenCalledWith(5);
        
        spy.mockRestore();
    });
});

// Testing with fixtures
describe('Test Fixtures', () => {
    const testUsers = [
        {id: 1, name: 'Alice', role: 'admin'},
        {id: 2, name: 'Bob', role: 'user'},
        {id: 3, name: 'Charlie', role: 'user'}
    ];
    
    test.each(testUsers)('user $name has valid role', (user) => {
        expect(['admin', 'user']).toContain(user.role);
    });
    
    // Reusable test data
    const fixtures = {
        validUser: {email: 'test@example.com', password: 'SecurePass123'},
        invalidUser: {email: 'invalid', password: 'short'},
        existingUser: {id: 1, email: 'existing@example.com'}
    };
    
    test('creates valid user', () => {
        expect(isValidUser(fixtures.validUser)).toBe(true);
    });
    
    test('rejects invalid user', () => {
        expect(isValidUser(fixtures.invalidUser)).toBe(false);
    });
});

// Testing with factories
class UserFactory {
    static create(overrides = {}) {
        return {
            id: Math.floor(Math.random() * 1000),
            name: 'Default Name',
            email: 'default@example.com',
            role: 'user',
            createdAt: new Date(),
            ...overrides
        };
    }
    
    static createAdmin(overrides = {}) {
        return this.create({role: 'admin', ...overrides});
    }
}

describe('UserFactory', () => {
    test('creates default user', () => {
        const user = UserFactory.create();
        
        expect(user.name).toBe('Default Name');
        expect(user.role).toBe('user');
    });
    
    test('creates admin user', () => {
        const user = UserFactory.createAdmin();
        
        expect(user.role).toBe('admin');
    });
    
    test('overrides properties', () => {
        const user = UserFactory.create({name: 'Custom', email: 'custom@example.com'});
        
        expect(user.name).toBe('Custom');
        expect(user.email).toBe('custom@example.com');
    });
});

// Performance testing
describe('Performance', () => {
    test('array operations are fast', () => {
        const largeArray = Array.from({length: 10000}, (_, i) => i);
        
        const start = performance.now();
        const result = largeArray.filter(x => x % 2 === 0).map(x => x * 2);
        const end = performance.now();
        
        expect(end - start).toBeLessThan(100); // Should complete in < 100ms
        expect(result.length).toBe(5000);
    });
});

// Testing with database transactions
describe('Database Tests', () => {
    let db;
    
    beforeEach(async () => {
        db = await Database.connect('test');
        await db.beginTransaction();
    });
    
    afterEach(async () => {
        await db.rollback(); // Rollback changes after each test
        await db.disconnect();
    });
    
    test('creates user in database', async () => {
        const user = await db.users.create({
            email: 'test@example.com',
            name: 'Test User'
        });
        
        expect(user.id).toBeDefined();
        
        const found = await db.users.findById(user.id);
        expect(found.email).toBe('test@example.com');
    });
});

// Snapshot testing for serialized data
describe('Snapshot Serialization', () => {
    test('complex object snapshot', () => {
        const data = {
            users: [
                {id: 1, name: 'John', preferences: {theme: 'dark'}},
                {id: 2, name: 'Jane', preferences: {theme: 'light'}}
            ],
            metadata: {
                total: 2,
                page: 1,
                timestamp: new Date('2024-01-15')
            }
        };
        
        // Custom serializer for dates
        expect.addSnapshotSerializer({
            test: (val) => val instanceof Date,
            print: (val) => val.toISOString()
        });
        
        expect(data).toMatchSnapshot();
    });
});
```

---

## **42.4 Other Unit Testing Frameworks Overview**

### **42.4.1 NUnit (.NET/C#)**

```csharp
// NUnit Test Example
using NUnit.Framework;
using Moq;
using System;
using System.Collections.Generic;

namespace CalculatorTests
{
    [TestFixture]
    public class CalculatorTests
    {
        private Calculator _calculator;
        
        [SetUp]
        public void Setup()
        {
            _calculator = new Calculator();
        }
        
        [TearDown]
        public void Cleanup()
        {
            _calculator = null;
        }
        
        [Test]
        public void Add_TwoNumbers_ReturnsSum()
        {
            // Arrange
            int a = 5;
            int b = 3;
            
            // Act
            int result = _calculator.Add(a, b);
            
            // Assert
            Assert.That(result, Is.EqualTo(8));
            Assert.AreEqual(8, result);
        }
        
        [Test]
        [TestCase(1, 2, 3)]
        [TestCase(5, 5, 10)]
        [TestCase(10, -2, 8)]
        public void Add_ParameterizedTests(int a, int b, int expected)
        {
            int result = _calculator.Add(a, b);
            Assert.AreEqual(expected, result);
        }
        
        [Test]
        public void Divide_ByZero_ThrowsException()
        {
            Assert.That(() => _calculator.Divide(10, 0), 
                Throws.TypeOf<DivideByZeroException>());
        }
        
        [Test]
        [Ignore("Not implemented yet")]
        public void FutureTest()
        {
            // Test skipped
        }
    }
    
    // Mocking with Moq
    [TestFixture]
    public class UserServiceTests
    {
        private Mock<IUserRepository> _mockRepo;
        private Mock<IEmailService> _mockEmail;
        private UserService _service;
        
        [SetUp]
        public void Setup()
        {
            _mockRepo = new Mock<IUserRepository>();
            _mockEmail = new Mock<IEmailService>();
            _service = new UserService(_mockRepo.Object, _mockEmail.Object);
        }
        
        [Test]
        public void CreateUser_ValidData_CreatesUser()
        {
            // Arrange
            var email = "test@example.com";
            var name = "Test User";
            
            _mockRepo.Setup(x => x.FindByEmail(email))
                     .Returns((User)null);
            
            _mockRepo.Setup(x => x.Save(It.IsAny<User>()))
                     .Returns(new User { Id = 1, Email = email, Name = name });
            
            // Act
            var result = _service.CreateUser(email, name);
            
            // Assert
            Assert.IsNotNull(result);
            Assert.AreEqual(1, result.Id);
            
            _mockRepo.Verify(x => x.FindByEmail(email), Times.Once);
            _mockRepo.Verify(x => x.Save(It.IsAny<User>()), Times.Once);
            _mockEmail.Verify(x => x.SendWelcome(email, name), Times.Once);
        }
    }
}
```

### **42.4.2 TestNG (Java Alternative)**

```java
// TestNG Example
import org.testng.annotations.*;
import org.testng.asserts.SoftAssert;
import static org.testng.Assert.*;

public class TestNGExample {
    
    private Calculator calculator;
    private SoftAssert softAssert;
    
    @BeforeClass
    public void setUpClass() {
        System.out.println("Before Class - runs once");
    }
    
    @AfterClass
    public void tearDownClass() {
        System.out.println("After Class - runs once");
    }
    
    @BeforeMethod
    public void setUp() {
        calculator = new Calculator();
        softAssert = new SoftAssert();
    }
    
    @AfterMethod
    public void tearDown() {
        calculator = null;
    }
    
    @Test
    public void testAddition() {
        assertEquals(calculator.add(2, 3), 5);
    }
    
    @Test
    public void testDivision() {
        assertEquals(calculator.divide(10, 2), 5.0);
    }
    
    @Test(expectedExceptions = ArithmeticException.class)
    public void testDivideByZero() {
        calculator.divide(10, 0);
    }
    
    @Test(dataProvider = "numberProvider")
    public void testWithDataProvider(int a, int b, int expected) {
        assertEquals(calculator.add(a, b), expected);
    }
    
    @DataProvider(name = "numberProvider")
    public Object[][] provideNumbers() {
        return new Object[][] {
            {1, 2, 3},
            {5, 5, 10},
            {10, -2, 8}
        };
    }
    
    @Test
    public void testSoftAssertions() {
        // Soft assertions continue after failure
        softAssert.assertEquals(calculator.add(2, 2), 4);
        softAssert.assertEquals(calculator.add(2, 2), 5); // Will fail but continue
        softAssert.assertEquals(calculator.add(2, 2), 4);
        
        softAssert.assertAll(); // Report all failures
    }
    
    @Test
    @Parameters({"a", "b", "expected"})
    public void testWithParameters(int a, int b, int expected) {
        assertEquals(calculator.add(a, b), expected);
    }
    
    @Test
    public void testTimeout() {
        // Test will fail if takes > 1000ms
        calculator.complexOperation(1000000);
    }
}
```

### **42.4.3 Mocha/Chai (JavaScript Alternative)**

```javascript
// Mocha with Chai Example
const { expect, assert, should } = require('chai');
should(); // Initialize should style

describe('Calculator', function() {
    let calculator;
    
    beforeEach(function() {
        calculator = new Calculator();
    });
    
    // Different assertion styles
    describe('Assertion Styles', function() {
        it('should work with expect style', function() {
            expect(calculator.add(2, 3)).to.equal(5);
            expect(calculator.add(2, 3)).to.be.a('number');
            expect(calculator.add.bind(calculator, 10, 0)).to.throw();
        });
        
        it('should work with assert style', function() {
            assert.equal(calculator.add(2, 3), 5);
            assert.typeOf(calculator.add(2, 3), 'number');
            assert.throws(() => calculator.divide(10, 0));
        });
        
        it('should work with should style', function() {
            calculator.add(2, 3).should.equal(5);
            calculator.add(2, 3).should.be.a('number');
            (() => calculator.divide(10, 0)).should.throw();
        });
    });
    
    // Async tests
    describe('Async Operations', function() {
        it('should handle promises (return promise)', function() {
            return fetchData().then(data => {
                expect(data).to.equal('peanut butter');
            });
        });
        
        it('should handle async/await', async function() {
            const data = await fetchData();
            expect(data).to.equal('peanut butter');
        });
        
        it('should handle callbacks (with done)', function(done) {
            fetchDataCallback((err, data) => {
                expect(err).to.be.null;
                expect(data).to.equal('peanut butter');
                done();
            });
        });
    });
    
    // Hooks
    before(function() {
        console.log('Before all tests');
    });
    
    after(function() {
        console.log('After all tests');
    });
    
    beforeEach(function() {
        console.log('Before each test');
    });
    
    afterEach(function() {
        console.log('After each test');
    });
    
    // Pending tests
    it('should implement this test');
    
    // Exclusive tests (only run these)
    it.only('only this test will run', function() {
        expect(true).to.be.true;
    });
    
    // Skipped tests
    it.skip('this test will be skipped', function() {
        expect(false).to.be.true;
    });
});
```

### **42.4.4 Framework Comparison Table**

| Feature | JUnit 5 | pytest | Jest | NUnit | TestNG |
|---------|---------|--------|------|-------|--------|
| **Language** | Java | Python | JavaScript | C#/.NET | Java |
| **Assertions** | Built-in | Built-in | Built-in | Built-in | Built-in |
| **Mocking** | Mockito | unittest.mock | jest.mock | Moq | Mockito/EasyMock |
| **Parameterized** | @ParameterizedTest | @pytest.mark.parametrize | test.each | [TestCase] | @DataProvider |
| **Parallel Execution** | Yes | pytest-xdist | --maxWorkers | Yes | Yes |
| **Test Discovery** | Automatic | Automatic | Automatic | Automatic | XML/Annotations |
| **Fixtures** | @BeforeEach/@AfterEach | @pytest.fixture | beforeEach/afterEach | [SetUp]/[TearDown] | @BeforeMethod/@AfterMethod |
| **Assertion Libraries** | Hamcrest, AssertJ | Built-in | Built-in | FluentAssertions | TestNG assertions |
| **IDE Support** | Excellent | Excellent | Good | Excellent | Excellent |
| **CI/CD Integration** | Maven/Gradle | pytest | Jest | dotnet test | Maven/Gradle |
| **Learning Curve** | Moderate | Easy | Easy | Moderate | Moderate |

---

## **Chapter Summary**

In this comprehensive chapter on **Unit Testing Frameworks**, we covered the three most important frameworks in modern software development:

**42.1 JUnit (Java):** The industry standard for Java testing with JUnit 5 (Jupiter). We explored:
- Annotations: `@Test`, `@BeforeEach`, `@AfterEach`, `@BeforeAll`, `@AfterAll`
- Assertions: `assertEquals`, `assertThrows`, `assertAll`, `assertTimeout`
- Advanced features: Parameterized tests, nested tests, dynamic tests, test templates
- Mocking with Mockito: `@Mock`, `@InjectMocks`, `@Captor`, verification, stubbing
- Test lifecycle management and conditional test execution

**42.2 pytest (Python):** Python's most powerful testing framework with minimal boilerplate:
- Simple test functions and test classes
- Fixture system with scopes (function, class, module, session)
- Parameterization with `@pytest.mark.parametrize`
- Mocking with `unittest.mock` and `monkeypatch`
- Exception testing, warnings, and floating-point approximations
- Async test support with `pytest-asyncio`

**42.3 Jest (JavaScript):** The comprehensive JavaScript testing framework:
- Test structure with `test()` and `describe()`
- Rich matchers: `toBe`, `toEqual`, `toContain`, `toMatch`, `toThrow`
- Async testing: callbacks, promises, async/await
- Mock functions: `jest.fn()`, mock implementations, spy functionality
- Timer mocking: `jest.useFakeTimers()`, `advanceTimersByTime()`
- Snapshot testing for UI components
- Module mocking and class mocking

**42.4 Other Frameworks:** Brief overview of NUnit (.NET), TestNG (Java), and Mocha/Chai (JavaScript) with framework comparison.

**Best Practices Covered:**
- Test isolation and independence
- Proper use of fixtures and setup/teardown
- Meaningful test names and organization
- Effective mocking strategies
- Testing edge cases and exceptions
- Parameterized tests for data-driven testing
- Code coverage targets and measurement

---

## **📖 Next Chapter: Chapter 43 - Behavior-Driven Development (BDD)**

Now that you have mastered unit testing frameworks across multiple languages, Chapter 43 will introduce you to **Behavior-Driven Development**, which extends unit testing with business-readable specifications.

In **Chapter 43**, you will learn:

- **BDD Fundamentals:** What is BDD and how it differs from TDD, focusing on business value and communication between stakeholders.

- **Gherkin Syntax:** The Given-When-Then format for writing feature files that both business users and technical teams can understand.

- **Cucumber Framework:** Implementing BDD with Cucumber in Java, including feature files, step definitions, hooks, and tags.

- **Behave (Python BDD):** Python's BDD framework with practical examples and integration with pytest.

- **Cypress with Cucumber:** Combining modern web testing with BDD specifications.

- **Robot Framework:** Keyword-driven testing approach for acceptance tests.

- **BDD Best Practices:** Writing effective scenarios, managing step definition reuse, and integrating BDD into CI/CD pipelines.

**Chapter 43 will transform how you think about testing—from verifying implementation details to specifying expected behavior in business terms.**